# Data Merging
---

In [1]:
%%time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from  datetime import datetime as dt
plt.style.use('fivethirtyeight')
%matplotlib inline

CPU times: user 1.05 s, sys: 297 ms, total: 1.35 s
Wall time: 1.61 s


In [42]:
df_temp = pd.read_csv ('../data/firstPass/GlobalTemperatures.csv', parse_dates = ['dt'])
df_temp = df_temp.rename (index = str, columns = {'LandAndOceanAverageTemperature': 'TempGlobal', 'LandAverageTemperature': 'TempLand'})
df_temp ['year'] = pd.DatetimeIndex (df_temp ['dt']).year
df_temp = df_temp.drop (['dt'], axis = 1)
df_aggTemp = df_temp.groupby ('year')[['TempGlobal', 'TempLand']].mean ()
df_aggTempLowerCut = df_aggTemp [df_aggTemp.index > 1899]
df_aggTempFinal = df_aggTempLowerCut [df_aggTempLowerCut.index < 2016]
df_aggTempFinal.index = df_aggTempFinal.index.map (unicode)

print
print '####### Decision Data Head'
print df_aggTempFinal.head ()
print
print '####### Decision Data Tail'
print df_aggTempFinal.tail ()
print
print '####### Decision Data Shape'
print df_aggTempFinal.shape



####### Decision Data Head
      TempGlobal  TempLand
1900   15.143917  8.501583
1901   15.073333  8.541917
1902   14.958333  8.304417
1903   14.836583  8.220167
1904   14.810417  8.090917

####### Decision Data Tail
      TempGlobal  TempLand
2011   15.769500  9.516000
2012   15.802333  9.507333
2013   15.854417  9.606500
2014   15.913000  9.570667
2015   16.058583  9.831000

####### Decision Data Shape
(116, 2)


## Add CO2 Data
---

In [43]:
df_co2 = pd.read_csv ('../data/co2/annual-co-emissions-per-country.csv')
df_co2 = df_co2.rename (index = str, columns = {'Annual CO₂ emissions (Global Carbon Project (2017)) (million tonnes)': 'CO2-Concentration', 'Year': 'year'})
df_aggCo2 = df_co2.groupby (df_co2 ['year'])[['CO2-Concentration']].sum ()
df_aggCo2LowerCut = df_aggCo2 [df_aggCo2.index > 1899]
df_aggCo2Final = df_aggCo2LowerCut [df_aggCo2LowerCut.index < 2016]
df_aggCo2Final.index = df_aggCo2Final.index.map (unicode)

print
print '####### CO-2 Data Head'
print df_aggCo2Final.head ()
print
print '####### CO-2 Data Tail'
print df_aggCo2Final.tail ()
print
print '####### CO-2 Data Shape'
print df_aggCo2Final.shape


####### CO-2 Data Head
      CO2-Concentration
1900         1880.29881
1901         1942.99709
1902         1996.90205
1903         2186.94199
1904         2202.66794

####### CO-2 Data Tail
      CO2-Concentration
2011          174562.07
2012          177452.11
2013          178642.90
2014          179957.65
2015          179639.07

####### CO-2 Data Shape
(116, 1)


In [44]:
# Merge Co2 and Temperature dataframes
df_final = df_aggCo2Final.join (df_aggTempFinal, how = 'inner')

print
print '******* MERGED DATAFRAME (P-1)'
print '####### Merged Data Head'
print df_final.head ()
print
print '####### Merged Data Tail'
print df_final.tail ()
print
print '####### Merged Data Shape'
print df_final.shape


******* MERGED DATAFRAME (P-1)
####### Merged Data Head
      CO2-Concentration  TempGlobal  TempLand
1900         1880.29881   15.143917  8.501583
1901         1942.99709   15.073333  8.541917
1902         1996.90205   14.958333  8.304417
1903         2186.94199   14.836583  8.220167
1904         2202.66794   14.810417  8.090917

####### Merged Data Tail
      CO2-Concentration  TempGlobal  TempLand
2011          174562.07   15.769500  9.516000
2012          177452.11   15.802333  9.507333
2013          178642.90   15.854417  9.606500
2014          179957.65   15.913000  9.570667
2015          179639.07   16.058583  9.831000

####### Merged Data Shape
(116, 3)


## Add Fossil Fuel Consumption Data
---

In [46]:
df_ffuels = pd.read_csv ('../data/fossilFuels/global-fossil-fuel-consumption.csv')
df_ffuels = df_ffuels.rename (index = str, columns = {'Year': 'year', 'Coal (terawatt-hours)': 'Coal', 'Crude oil (terawatt-hours)': 'Oil', 'Natural gas (terawatt-hours)': 'Gas'})
df_aggFfuels = df_ffuels.groupby (df_ffuels ['year'])[['Coal', 'Oil', 'Gas']].sum ()
df_ffuelsLowerCut = df_aggFfuels [df_aggFfuels.index > 1899]
df_ffuelsFinal = df_ffuelsLowerCut [df_ffuelsLowerCut.index < 2016]
df_ffuelsFinal.index = df_ffuelsFinal.index.map (unicode)

print
print '####### F-Fuels Data Head'
print df_ffuelsFinal.head ()
print
print '####### F-Fuels Data Tail'
print df_ffuelsFinal.tail ()
print
print '####### F-Fuels Data Shape'
print df_ffuelsFinal.shape


####### F-Fuels Data Head
             Coal         Oil        Gas
1900   5727.78236   180.55570   63.88894
1910   8655.56248   397.22254  141.66678
1920   9833.34120   888.88960  233.33352
1930  10125.00810  1755.55696  602.77826
1940  11586.12038  2652.77990  875.00070

####### F-Fuels Data Tail
             Coal          Oil          Gas
2011  44277.59112  47982.28000  34033.08509
2012  44395.06224  48569.02647  35012.27319
2013  45205.49828  49088.50592  35522.26380
2014  45233.99218  49483.64369  35738.83055
2015  44015.53033  50485.32693  36596.65882

####### F-Fuels Data Shape
(58, 3)


# PROBLEM
---
We have only **58 values** for fossil-fuel datasets in the desired time-range ([1900 - 2015]). We expect **116** values.

## Work-Around
----
I am going to do partial mergin at the moment. Missing years from Fossil datasets will be treated as *NA*.

In [58]:
# Add fossil-fuel consumption dataset to partially merged frame
df_final2 = df_ffuelsFinal.join (df_final, how = 'outer')

print
print '******* MERGED DATAFRAME (P-2)'
print '####### Merged Data Head'
print df_final2.head ()
print
print '####### Merged Data Tail'
print df_final2.tail ()
print
print '####### Merged Data Shape'
print df_final2.shape


******* MERGED DATAFRAME (P-2)
####### Merged Data Head
            Coal       Oil       Gas  CO2-Concentration  TempGlobal  TempLand
1900  5727.78236  180.5557  63.88894         1880.29881   15.143917  8.501583
1901         NaN       NaN       NaN         1942.99709   15.073333  8.541917
1902         NaN       NaN       NaN         1996.90205   14.958333  8.304417
1903         NaN       NaN       NaN         2186.94199   14.836583  8.220167
1904         NaN       NaN       NaN         2202.66794   14.810417  8.090917

####### Merged Data Tail
             Coal          Oil          Gas  CO2-Concentration  TempGlobal  \
2011  44277.59112  47982.28000  34033.08509          174562.07   15.769500   
2012  44395.06224  48569.02647  35012.27319          177452.11   15.802333   
2013  45205.49828  49088.50592  35522.26380          178642.90   15.854417   
2014  45233.99218  49483.64369  35738.83055          179957.65   15.913000   
2015  44015.53033  50485.32693  36596.65882          179639

## Add Sea-Level Data
---

In [54]:
df_seaLev = pd.read_csv ('../Sea_level/Data/sea-level1900.csv')
df_seaLev = df_seaLev.rename (index = str, columns = {'GMSL': 'SeaLevel'})
df_aggSeaLev = df_seaLev.groupby (df_seaLev ['year'])[['SeaLevel']].sum ()
df_seaLevLowerCut = df_aggSeaLev [df_aggSeaLev.index > 1899]
df_seaLevFinal = df_seaLevLowerCut [df_seaLevLowerCut.index < 2016]
df_seaLevFinal.index = df_seaLevFinal.index.map (unicode)

print
print '####### Sea-Level Data Head'
print df_seaLevFinal.head ()
print
print '####### Sea-Level Data Tail'
print df_seaLevFinal.tail ()
print
print '####### Sea-Level Data Shape'
print df_seaLevFinal.shape


####### Sea-Level Data Head
      SeaLevel
1900    -130.1
1901    -130.5
1902    -125.9
1903    -117.9
1904    -128.2

####### Sea-Level Data Tail
      SeaLevel
2009      58.0
2010      65.7
2011      67.3
2012      76.1
2013      67.7

####### Sea-Level Data Shape
(114, 1)


In [65]:
# Add sea-level dataset to partially merged frame
df_final3 = df_seaLevFinal.join (df_final2, how = 'outer')

print
print '******* MERGED DATAFRAME (P-2)'
print '####### Merged Data Head'
print df_final3.head ()
print
print '####### Merged Data Tail'
print df_final3.tail ()
print
print '####### Merged Data Shape'
print df_final3.shape
print


******* MERGED DATAFRAME (P-2)
####### Merged Data Head
      SeaLevel        Coal       Oil       Gas  CO2-Concentration  TempGlobal  \
1900    -130.1  5727.78236  180.5557  63.88894         1880.29881   15.143917   
1901    -130.5         NaN       NaN       NaN         1942.99709   15.073333   
1902    -125.9         NaN       NaN       NaN         1996.90205   14.958333   
1903    -117.9         NaN       NaN       NaN         2186.94199   14.836583   
1904    -128.2         NaN       NaN       NaN         2202.66794   14.810417   

      TempLand  
1900  8.501583  
1901  8.541917  
1902  8.304417  
1903  8.220167  
1904  8.090917  

####### Merged Data Tail
      SeaLevel         Coal          Oil          Gas  CO2-Concentration  \
2011      67.3  44277.59112  47982.28000  34033.08509          174562.07   
2012      76.1  44395.06224  48569.02647  35012.27319          177452.11   
2013      67.7  45205.49828  49088.50592  35522.26380          178642.90   
2014       NaN  45233.99

# Final Dataset
---

In [66]:
df_final3.head ()

,SeaLevel,Coal,Oil,Gas,CO2-Concentration,TempGlobal,TempLand
1900,-130.1,5727.78236,180.5557,63.88894,1880.29881,15.143917,8.501583
1901,-130.5,NaN,NaN,NaN,1942.99709,15.073333,8.541917
1902,-125.9,NaN,NaN,NaN,1996.90205,14.958333,8.304417
1903,-117.9,NaN,NaN,NaN,2186.94199,14.836583,8.220167
1904,-128.2,NaN,NaN,NaN,2202.66794,14.810417,8.090917


**NOTE**: We have to do something about the missing values in **fossil-fuel** dataset.
At the moment, I am using **forward-fill** method to replace NaN fossil-fuel values. However, it would be best if we can get actual data for these values.

In [67]:
df_final4 = df_final3.fillna (method = 'ffill')
df_final4.head (5)

,SeaLevel,Coal,Oil,Gas,CO2-Concentration,TempGlobal,TempLand
1900,-130.1,5727.78236,180.5557,63.88894,1880.29881,15.143917,8.501583
1901,-130.5,5727.78236,180.5557,63.88894,1942.99709,15.073333,8.541917
1902,-125.9,5727.78236,180.5557,63.88894,1996.90205,14.958333,8.304417
1903,-117.9,5727.78236,180.5557,63.88894,2186.94199,14.836583,8.220167
1904,-128.2,5727.78236,180.5557,63.88894,2202.66794,14.810417,8.090917


In [70]:
df_final4.index.name = 'year'

# Save the final dataframe to csv file
df_final4.to_csv ('../data/mergedData.csv')